In [1]:
import math
from compas.geometry import Point, Vector
from compas.datastructures import Mesh
from compas.geometry import Translation, transform_points, Rotation

# ==============================================================================
# 幾何輔助函式 (回傳原始座標列表)
# ==============================================================================

def get_rotated_quad(center: Point, width_x: float, width_y: float, angle: float) -> list[list[float]]:
    """
    計算一個旋轉後的矩形底面（四個頂點），回傳原始座標列表 [[x, y, z], ...]。
    """
    half_x, half_y = width_x * 0.5, width_y * 0.5
    
    # 1. 建立在原點的基礎頂點
    base_pts = [
        Point( half_x,  half_y, 0),
        Point(-half_x,  half_y, 0),
        Point(-half_x, -half_y, 0),
        Point( half_x, -half_y, 0),
    ]
    
    # 2. 執行旋轉 (繞 Z 軸)
    R = Rotation.from_axis_and_angle(Vector(0, 0, 1), angle)
    # transform_points 回傳原始座標列表
    rotated_coords = transform_points(base_pts, R) 
    
    # 3. 執行平移 (移動到目標中心點)
    T = Translation.from_vector(Vector(center.x, center.y, center.z))
    # transform_points 輸入和輸出都是原始座標列表
    final_coords = transform_points(rotated_coords, T) 

    # 4. 直接回傳原始座標列表
    return final_coords


def create_twisted_prism_mesh(base_quad_coords: list, top_quad_coords: list) -> Mesh:
    """
    從底部和頂部四邊形的原始座標列表建立一個 Mesh 物件。
    """
    
    # 頂點列表是兩個原始座標列表的合併
    vertices = base_quad_coords + top_quad_coords 
    
    # ✅ 最終穩定的座標轉換 (確認與您的輸出一致)
    vertex_coords = [ tuple(p) for p in vertices ]
    
    # 面列表 (底部、頂部、四個側面)
    faces = [
        [0, 1, 2, 3],    # 底部
        [7, 6, 5, 4],    # 頂部
        [0, 3, 7, 4],    # 側面 1
        [3, 2, 6, 7],    # 側面 2
        [2, 1, 5, 6],    # 側面 3
        [1, 0, 4, 5],    # 側面 4
    ]
    
    # 直接使用原始座標列表創建 Mesh
    return Mesh.from_vertices_and_faces(vertex_coords, faces)


# ==============================================================================
# 核心生成函式
# ==============================================================================

def create_single_twisted_prism(angle_deg: int, base_x: float, base_y: float, height: float) -> Mesh:
    """
    生成一個指定扭轉角度的單一矩形塊體，只回傳 Mesh 物件。
    """
    
    angle_rad = math.radians(angle_deg)
    
    # 底部中心點 (Z=0)
    base_center = Point(0, 0, 0) 
    # 頂部中心點 (Z=height)
    top_center = Point(0, 0, height) 
    
    # 底部頂點 (旋轉角度為 0) - 回傳座標列表
    base_quad_coords = get_rotated_quad(base_center, base_x, base_y, angle=0.0)
    
    # 頂部頂點 (旋轉角度為 angle_rad) - 回傳座標列表
    top_quad_coords = get_rotated_quad(top_center, base_x, base_y, angle=angle_rad)
    
    return create_twisted_prism_mesh(base_quad_coords, top_quad_coords)

In [3]:
from compas_notebook.viewer import Viewer
from compas.geometry import Translation
from compas.datastructures import Mesh

combined_mesh = Mesh()

columns = 20
spacing = 15.0
print("正在計算幾何數據...")

for i in range(360):
    temp_mesh = create_single_twisted_prism(
        angle_deg=i, 
        base_x=5.0, 
        base_y=5.0, 
        height=10.0
    )
    
    col = i % columns
    row = i // columns
    T = Translation.from_vector([col * spacing, row * spacing, 0])
    

    temp_mesh.transform(T)
    
    combined_mesh.join(temp_mesh)


viewer = Viewer()
viewer.scene.add(combined_mesh, name="All_Twisted_Prisms", show_edges=True)
viewer.show()

正在計算幾何數據...
